<a href="https://colab.research.google.com/github/nigoda/machine_learning/blob/main/16_Text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.tensorflow.org/tutorials/keras/text_classification

In [ ]:
!pip install -q tensorflow-hub

In [ ]:
import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ",tf.__version__)
print("Eager mode: ",tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is","available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.1.0
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


### **Download the IMDB dataset**

Split the training set into 60% and 40%, so we'll end up with 15,000 examples for training, 10,000 examples for validation and 25,000 examples for testing.

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8Y4F45/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8Y4F45/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8Y4F45/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


### **Explore the data**

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

Build the model

The neural network is created by stacking layers-this requires three main architectural decision:

1.   How to represent the text?
2.   How many layers to use in the model
3.   How many hidden units to use for each layer?

In this example, the input data consist of sentences. The labels to predict are either 0 or 1.

One way to represent the text is to convert sentences into embedding vectors, we can use a pre-trained text enbedding as the first layer, which will have three advantages:
1.   We don't have to worry about text preprocessing,
2.   We can benefit from transfer learning,
3.   The embedding has a fixed size, so it's simpler to process.

For this example we will use a pre-trained text embedding model from [TensorFlow Hub](https://tfhub.dev/) called [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2).

There are many other pre-trained text embeddings from TFHub that can be used in this tutorial:

1.  [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - trained with the same NNLM architecture on the same data as [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2), but with a larger embedding dimension. Larger dimensional embeddings can improve on your task but it may take longer to train your model.

2.  [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - the same as [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2), but with additional text normalization such as removing punctuation. This can help if the text in your task contains additional characters or punctuation.

3.  [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) - a much larger model yielding 512 dimensional embeddings trained with a deep averaging network (DAN) encoder.

And many more! Find more [text embedding models](https://tfhub.dev/s?module-type=text-embedding) on TFHub.



In [ ]:
embedding = "http://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

let's now build the full model:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


The layer are stacked sequentially to build the classifier:

1.  The first layer is a TensorFlow Hub layer. This layer uses pre-trained Saved Model to map a sentence into its embedding vector. The pre-trained text embedding model that we are using splits the sentence into tokens, embeds each token and then combines the embedding. The resulting dimensions are: (`num_examples, embedding_dimension`). For this NNLM model, the `embedding_dimension` is 50.

2.  This fixed-length output vector is piped through a fully-connected(Dense) layer with 16 hidden units.

3.  The last layer is densely conneted with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probabilty, or confidence level.

Let's compile the model.

### **Loss function and optimizer**

A model needs a loss function and an optimizer for training. Since this is a binary classification problem and the model outputs logits(a single-unit layer with a linear activation), we'll use the `binary_crossentropy` loss function.

This isn't the only choice for a loss function, you could, for instance, choose `mean_squared_error`. But,generally,`binary_crossentropy` is better for dealing with probabilities-it measures the "distance" between probability distributions, or in our case, between the ground-truth distributiona and the predictions.

Later, when we are exploring regression problems(say, to predict the price of a house), we will see how to use another loss function called mean squared error.

Now, configure the model to use an optimizer and loss function:

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

### **Train the model**

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 27s 916ms/step - loss: 0.6771 - accuracy: 0.5261 - val_loss: 0.6353 - val_accuracy: 0.6134
Epoch 2/10
30/30 [==============================] - 20s 672ms/step - loss: 0.5767 - accuracy: 0.6701 - val_loss: 0.5327 - val_accuracy: 0.7219
Epoch 3/10
30/30 [==============================] - 20s 672ms/step - loss: 0.4425 - accuracy: 0.7979 - val_loss: 0.4268 - val_accuracy: 0.8026
Epoch 4/10
30/30 [==============================] - 20s 671ms/step - loss: 0.3255 - accuracy: 0.8725 - val_loss: 0.3598 - val_accuracy: 0.8379
Epoch 5/10
30/30 [==============================] - 20s 667ms/step - loss: 0.2398 - accuracy: 0.9126 - val_loss: 0.3280 - val_accuracy: 0.8605
Epoch 6/10
30/30 [==============================] - 20s 665ms/step - loss: 0.1762 - accuracy: 0.9410 - val_loss: 0.3140 - val_accuracy: 0.8631
Epoch 7/10
30/30 [==============================] - 20s 668ms/step - loss: 0.1279 - accuracy: 0.9617 - val_loss: 0.3110 - val_accuracy: 0.8679

### **Evaluate the model**

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.362
accuracy: 0.847


This fairly naive approach achieves an accuracy of about 87%. With more advanced approaches, the model should get closer to 95%